In [1]:
import pybamm as pb;import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;#import imageio
from scipy.io import savemat,loadmat;from pybamm import constants,exp;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
for k in range(0,1):
    mpl.rcParams["axes.labelsize"] = fs
    mpl.rcParams["axes.titlesize"] = fs
    mpl.rcParams["xtick.labelsize"] =  fs
    mpl.rcParams["ytick.labelsize"] =  fs
    mpl.rcParams["legend.fontsize"] =  fs
    mpl.rcParams['font.sans-serif'] = ['Times New Roman']
    mpl.rcParams['font.serif'] = ['Times New Roman']
    mpl.rcParams['axes.unicode_minus'] = False
# Create GIF
def create_gif(image_list, gif_name, ds):
    frames = []
    for image_name in image_list:
        if image_name.endswith('.png'):
            print(image_name)
            frames.append(imageio.imread(image_name))
    # Save them as frames into a gif
    imageio.mimsave(gif_name, frames, 'GIF', duration = ds)
    return
import openpyxl


In [ ]:
# This code aims to teach me how to write, append, and read .xlsx files. 
# Ref link: https://www.cnblogs.com/wulixia/p/11937989.html

In [3]:
# New target: write excel for HPC:
# 1. before sweep, write all para into the summary file
# 2. after finish, manually all seperate successful excel into the target ones  

import sys  
str_path_0 = os.path.abspath(os.path.join(pb.__path__[0],'..'))
str_path_1 = os.path.abspath(os.path.join(str_path_0,"wip\Rio_Code\Fun_P2"))
sys.path.append(str_path_1) 
from Fun_P2 import (
    GetScan,
    recursive_scan,
    Para_init,
    Run_model_wwo_dry_out ,
    Cal_new_con_Update,
    Run_Model_Base_On_Last_Solution,
    Run_Model_Base_On_Last_Solution_RPT,
    write_excel_xlsx,
    Get_initial_cap,
    Get_initial_cap2,
    handle_signal,
    Run_P2_till_Fail,
)

########################  Input  ########################
# all values here must be a list, even it is a single object
Para_dict_All = {
   "Total ageing cycles":[234,1170],
   "Ageing cycles between RPT":[78,],
   "Update cycles for ageing":[13,],
   "Cycles within RPT":[2,],
   "Ageing temperature":[25,],
   "RPT temperature":[25,],
   "Particle mesh points":[30,],   # Simon uses 30
   #"Exponential mesh stretch":[1.0],
   "Para_Set":[ "Li2023_Coupled",],
   "Model option":[
         {
            "calculate discharge energy":"true",
            "particle": "Fickian diffusion",          
            "SEI":"ec reaction limited",   
            #"SEI on cracks":"true",  
            "SEI film resistance":"distributed",          
            "SEI porosity change":"true",      
            #"particle mechanics":("swelling and cracking", "swelling only"), 
            #"loss of active material":"stress-driven", 
            "lithium plating":"partially reversible"      },
         ],
   "Inner SEI reaction proportion":[0.5,],
   "Ratio of lithium moles to SEI moles":[2,], # I have always been using 1 for solvent consumption model
   "Initial inner SEI thickness [m]":[2.5E-9,],
   "Initial outer SEI thickness [m]":[2.5E-9,],
   "SEI growth activation energy [J.mol-1]":[38000,],
   # Solvent consumption sub-model
   "Initial electrolyte excessive amount ratio":[2,], # set to <1 for DryOut=Off 
   "Current solvent concentration in the reservoir [mol.m-3]":[4541.0,],
   "Current electrolyte concentration in the reservoir [mol.m-3]":[1000,],
   "Ratio of Li-ion concentration change in electrolyte consider solvent consumption":[1.0,],
   # DFN parameter
   "Upper voltage cut-off [V]":[4.21,],
   "Lower voltage cut-off [V]":[2.49,],

   # interstitial-diffusion limited
   'Inner SEI lithium interstitial diffusivity [m2.s-1]':[1e-12,],    
   'Lithium interstitial reference concentration [mol.m-3]':[15,],
   # ec-reaction limited
   'EC diffusivity [m2.s-1]':[1e-22,1e-23,1e-24,1e-26,1e-27,],
   'SEI kinetic rate constant [m.s-1]':[1e-12,], 
   'EC initial concentration in electrolyte [mol.m-3]':[4541.0,],
   # LiP and coupling with SEI:
   "Dead lithium decay constant [s-1]":[ 1e-7,],            # default: 1e-6
   'Lithium plating kinetic rate constant [m.s-1]':[3E-10,], # default: 1e-9
   # Crack model
   "Negative electrode LAM constant proportional term [s-1]":[ 2.7778e-9,], # default: 2.7778e-07
   "Positive electrode LAM constant proportional term [s-1]":[ 2.7778e-9,], # default: 2.7778e-07
   # make it simple for now,], but may want to have T dependency in the future
   "Negative electrode cracking rate":[ 3.9e-22,],   # default: function, ~3.9e-20
   "Positive electrode cracking rate":[ 3.9e-22,],   # default: function, ~3.9e-20
   "Negative electrode volume change":[ 0.0,],
   "Positive electrode volume change":[ 0.0,],
   "Initial Neg SOC":[0.850],    #list(np.linspace(0.84,0.90,6)),
   "Initial Pos SOC":[0.2705], # list(np.linspace(0.22,0.27,6)),

}
Para_dict_list = []
recursive_scan(Para_dict_list,Para_dict_All, list(Para_dict_All.keys()), {})
print(f"Total scan case is {len(Para_dict_list)}")

Total scan case is 10


In [4]:
# Wrtie head
BasicPath = 'D:/OneDrive - Imperial College London/SimDataSave/P2R7'; 
#BasicPath=os.getcwd()
Target  = '/Test_new_write/'
if not os.path.exists(BasicPath + Target):
   os.mkdir(BasicPath + Target);
book_name_xlsx = 'Test_new_write.xlsx';

sheet_name_xlsx = 'Results';

Path_pack = [BasicPath,Target,book_name_xlsx,sheet_name_xlsx,];



Successfully create a excel file


In [24]:
# Write para - 1st round:
head_list = list(Para_dict_list[0].keys())
head_list.insert(0,"Index");
head_list.insert(1,"Dry out");
head_list.extend([ "exp_AGE_text", "exp_RPT_text",
   "Cap Loss","LLI to LiP",
   "LLI to SEI","LLI to sei-on-cracks",
   "LAM to Neg","LAM to Pos",
   "Vol_Elely_Tot Final", "Vol_Elely_JR Final","Width Final","Error"])
# 
Values_1 = [head_list,];
index_list = np.arange(1,len(Para_dict_list)+1,1)
for Scan_i,Para_dict_i in zip(index_list,Para_dict_list):
    value_list_temp = list(Para_dict_i.values())
    values = []
    for value_list_temp_i in value_list_temp:
        values.append(str(value_list_temp_i))
    values.insert(0,str(Scan_i))
    Values_1.append(values)
write_excel_xlsx(
    BasicPath + Target+book_name_xlsx, 
    sheet_name_xlsx, Values_1)   


Successfully create a excel file


In [26]:
Index_List_succeed = [1,2];sheet_name_xlsx = 'Results';
for index_i in Index_List_succeed:
    try:
        old_book = str(index_i) + '_' + book_name_xlsx
        data_old = openpyxl.load_workbook(BasicPath + Target + old_book)   
        data_tar = openpyxl.load_workbook(BasicPath + Target + book_name_xlsx) 

        table_old = data_old[str(index_i)]
        #print(table_old)
        nrows_old = table_old.max_row  # 获得行数
        ncolumns_old = table_old.max_column  # 获得列数


        table_tar = data_tar[sheet_name_xlsx]
        #print(table_tar)
        nrows_tar = index_i # Mark!!! Most important changes!
        ncolumns_old = table_old.max_column  # 获得列数
        #print(nrows_tar,ncolumns_old)
        list_old = [];
        # print(nrows_old,nrows_tar)
        for i in range(1,nrows_old+1):
            for j in range(1,ncolumns_old+1):
                list_old.append(table_old.cell(row=i,column=j).value)
        
        list_old = [list_old,]
        for i in range(1, len(list_old)+1):
            for j in range(1, len(list_old[i-1])+1):
                #print(i,j,list_old[i-1][j-1]    )
                table_tar.cell(nrows_tar+i, j).value = list_old[i-1][j-1]     
        data_tar.save(BasicPath + Target + book_name_xlsx) 
        data_tar.close()
    except:
        print(f"Something goes wrong for Scan {index_i}!")
    else:
        print(f"Successfuly write results for Scan {index_i}!") 


Successfuly write results for Scan 1!
Successfuly write results for Scan 2!


In [25]:
values_extra_1 = Values_1[1]
values_extra_1.extend([5,0.5])
values_extra_2 = Values_1[2]
values_extra_2.extend([4,0.4])

Succeed_index_1 = 1
Succeed_index_2 = 2
book_name_xlsx_seperate =   str(Succeed_index_1)+ '_' + book_name_xlsx;
sheet_name_xlsx =  str(Succeed_index_1);
write_excel_xlsx(
    BasicPath + Target+book_name_xlsx_seperate, 
    sheet_name_xlsx, [values_extra_1,])
book_name_xlsx_seperate =   str(Succeed_index_2)+ '_' + book_name_xlsx;
sheet_name_xlsx =  str(Succeed_index_2);
write_excel_xlsx(
    BasicPath + Target+book_name_xlsx_seperate, 
    sheet_name_xlsx, [values_extra_2,])
# write para and results - 2nd round:

Successfully create a excel file
Successfully create a excel file


In [2]:
def write_excel_xlsx(path, sheet_name, value):
    index = len(value)
    workbook = openpyxl.Workbook()  # 新建工作簿（默认有一个sheet？）
    sheet = workbook.active  # 获得当前活跃的工作页，默认为第一个工作页
    sheet.title = sheet_name  # 给sheet页的title赋值
    for i in range(0, index):
        for j in range(0, len(value[i])):
            sheet.cell(row=i + 1, column=j + 1, value=str(value[i][j]))  # 行，列，值 这里是从1开始计数的
    workbook.save(path)  # 一定要保存
    print("xlsx格式表格写入数据成功！")
 


In [ ]:

data = openpyxl.load_workbook('xlsx格式测试工作簿.xlsx')
 
# 取第一张表
sheetnames = data.get_sheet_names()
table = data.get_sheet_by_name(sheetnames[0])
 
table = data.active
print(table.title)  # 输出表名
nrows = table.max_row  # 获得行数
ncolumns = table.max_column  # 获得列数
values = [['E', 'X', 'C', 'E', 'L'],
          [1, 2, 3, 4, 5],
          ['a', 'b', 'c', 'd', 'e']]
# 注意行业列下标是从1开始的
for i in range(1, len(values)+1):
    for j in range(1, len(values[i-1])+1):
        table.cell(nrows+i, j).value = values[i-1][j-1]
 
data.save('xlsx格式测试工作簿.xlsx')
 
# values = ['E', 'X', 'C', 'E', 'L']
# for value in values:
#     table.cell(nrows+1, 1).value = value
#     nrows = nrows + 1

In [21]:
BasicPath = 'D:/OneDrive - Imperial College London/PhD Study/Imperial Study/Model/Pybamm Learning/SimDataSave/2nd paperTest R1/';
Target  = 'Big scan/'
if not os.path.exists(BasicPath + Target):
    os.mkdir(BasicPath + Target);
book_name_xlsx = 'Scan Chen2020_coupled 1170 cycles.xlsx'

sheet_name_xlsx = 'Results'
 
value3 = [
    ["Index", "cs_Neg_Init", "Diff_SEI", "R_SEI", 
    "Bulk_Sol_Con","D_Li_inSEI", "c_Li_inte_ref",
    "Couple_SEI_LiP","k_LiP","Cap Loss","LLI to LiP",
    "LLI to SEI","Vol_Elely_Tot Final", "Vol_Elely_JR Final","Width Final"],
    ]
write_excel_xlsx(BasicPath + Target+book_name_xlsx, sheet_name_xlsx, value3)

xlsx格式表格写入数据成功！


In [28]:
data = openpyxl.load_workbook(BasicPath + Target+'Scan Chen2020_coupled 1170 cycles.xlsx')
 
table = data.get_sheet_by_name('Results')
Diff_SEI = 1.700000000000002e-20; R_SEI = 2E5
print(table.title)  # 输出表名
nrows = table.max_row  # 获得行数
ncolumns = table.max_column  # 获得列数
values = [
    ["1", "28920", Diff_SEI, R_SEI, 
    "4541","1e-20", "15",
    "2E-6","1E-10","5","1",
    "2","3.2", "3.0","1.5"],
    ]
# 注意行业列下标是从1开始的
for i in range(1, len(values)+1):
    for j in range(1, len(values[i-1])+1):
        table.cell(nrows+i, j).value = values[i-1][j-1]
 
data.save(BasicPath + Target+'Scan Chen2020_coupled 1170 cycles.xlsx')

Results


C:\Users\LRH\AppData\Local\Temp/ipykernel_7376/2638982180.py:3: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  table = data.get_sheet_by_name('Results')


In [ ]:
#使用python在excel表格中增加新的sheet表 https://blog.csdn.net/zh54b5n64vn64654/article/details/81101681
import openpyxl
wb=openpyxl.load_workbook(r'ex.xlsx')
wb.create_sheet(title='Sheet2',index=0)
wb.save(r'ex.xlsx')